# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [4]:
#import numpy and pandas

import pandas as pd
import numpy as np

import scipy.stats as st


# Challenge 1 - The `stats` Submodule

This submodule contains statistical functions for conducting hypothesis tests, producing various distributions and other useful tools. Let's examine this submodule using the KickStarter dataset. Load the data using the `ks-projects-201801.csv` file

In [5]:
# Your code here:
kickstart = pd.read_csv('..\ks-projects-201801.csv')

Now print the `head` function to examine the dataset.

In [6]:
# Your code here:

kickstart.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


Import the `mode` function from `scipy.stats` and find the mode of the `country` and `currency` column.

In [7]:
# Your code here:
from scipy.stats import mode 

print(mode(kickstart['country']))
print(mode(kickstart['currency']))


ModeResult(mode=array(['US'], dtype=object), count=array([292627]))
ModeResult(mode=array(['USD'], dtype=object), count=array([295365]))


The trimmed mean is a function that computes the mean of the data with observations removed. The most common way to compute a trimmed mean is by specifying a percentage and then removing elements from both ends. However, we can also specify a threshold on both ends. The goal of this function is to create a more robust method of computing the mean that is less influenced by outliers. SciPy contains a function called `tmean` for computing the trimmed mean. 

In the cell below, import the `tmean` function and then find the 75th percentile of the `goal` column. Compute the trimmed mean between 0 and the 75th percentile of the column. Read more about the `tmean` function [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.tmean.html#scipy.stats.tmean).

In [8]:
# Your code here:

from scipy.stats import tmean

print(tmean(kickstart['goal'],limits = (0,len(kickstart)*.75)))


16264.213857704375


#### SciPy contains various statistical tests. One of the tests is Fisher's exact test. This test is used for contingency tables. 

The test originates from the "Lady Tasting Tea" experiment. In 1935, Fisher published the results of the experiment in his book. The experiment was based on a claim by Muriel Bristol that she can taste whether tea or milk was first poured into the cup. Fisher devised this test to disprove her claim. The null hypothesis is that the treatments do not affect outcomes, while the alternative hypothesis is that the treatment does affect outcome. To read more about Fisher's exact test, see:

* [Wikipedia's explanation](http://b.link/test61)
* [A cool deep explanation](http://b.link/handbook47)
* [An explanation with some important Fisher's considerations](http://b.link/significance76)

Let's perform Fisher's exact test on our KickStarter data. We intend to test the hypothesis that the choice of currency has an impact on meeting the pledge goal. We'll start by creating two derived columns in our dataframe. The first will contain 1 if the amount of money in `usd_pledged_real` is greater than the amount of money in `usd_goal_real`. We can compute this by using the `np.where` function. If the amount in one column is greater than the other, enter a value of 1, otherwise enter a value of zero. Add this column to the dataframe and name it `goal_met`.

In [9]:
# Your code here:
kickstart['goal_met'] = np.where(kickstart['usd_pledged_real']>kickstart['usd_goal_real'],1,0)


Next, create a column that checks whether the currency of the project is in US Dollars. Create a column called `usd` using the `np.where` function where if the currency is US Dollars, assign a value of 1 to the row and 0 otherwise.

In [10]:
# Your code here:

kickstart['usd'] =np.where(kickstart['currency']=='USD',1,0)

Now create a contingency table using the `pd.crosstab` function in the cell below to compare the `goal_met` and `usd` columns.

In [11]:
# Your code here:
cont_tab = pd.crosstab(kickstart['goal_met'], kickstart['usd'])
cont_tab


usd,0,1
goal_met,,
0,58685,187130
1,24611,108235


Import the `fisher_exact` function from `scipy.stats` and conduct the hypothesis test on the contingency table that you have generated above. You can read more about the `fisher_exact` function [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.fisher_exact.html#scipy.stats.fisher_exact). The output of the function should be the odds ratio and the p-value. The p-value will provide you with the outcome of the test.

In [12]:
# Your code here:
from scipy.stats import fisher_exact

alpha = .05 

stat,p_value = st.fisher_exact(cont_tab)
print(stat,p_value)
print(p_value<alpha)
print("Pvalue is lower than alpha. Currency does not have impact on pledge goal")

1.3791838163150314 2.994e-320
True
Pvalue is lower than alpha. Currency does not have impact on pledge goal


# Challenge 2 - The `interpolate` submodule

This submodule allows us to interpolate between two points and create a continuous distribution based on the observed data.

In the cell below, import the `interp1d` function from `scipy.interpolate` and first take a sample of 10 rows from `kickstarter`. 

**Make sure there are not duplicated values in `backers`**

In [13]:
# Your code here:

from scipy.interpolate import interp1d 

kick_sample = kickstart.sample(10)
kick_sample



,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,goal_met,usd
168226,1855845632,J-World: Welcome to imagiNation,Mobile Games,Games,USD,2017-10-15,10000.0,2017-08-16 00:15:06,25.0,failed,1,US,0.00,25.00,10000.00,0,1
255876,37113472,The River Rock Band--Drawn From the Well. Stud...,Faith,Music,USD,2016-03-15,5500.0,2016-02-09 17:59:20,5555.0,successful,69,US,5555.00,5555.00,5500.00,1,1
301135,603358126,BE's Production Film,Experimental,Film & Video,USD,2015-07-07,1000.0,2015-06-07 07:42:35,5.0,failed,1,US,5.00,5.00,1000.00,0,1
116502,1591850155,Primal Hero - Online Browser-Based Monster Fig...,Video Games,Games,USD,2014-02-12,40000.0,2014-01-13 23:20:24,1973.0,failed,36,US,1973.00,1973.00,40000.00,0,1
131160,1665771453,Flowers By Number | DIY Flower Arrangements,DIY,Crafts,USD,2015-12-01,20000.0,2015-11-01 10:44:56,13914.0,failed,50,US,13914.00,13914.00,20000.00,0,1
183960,1936618996,"Chubbys BBQ Food Truck- Portland, Maine",Food Trucks,Food,USD,2015-04-09,17500.0,2015-03-10 02:03:46,216.0,failed,4,US,216.00,216.00,17500.00,0,1
65661,1333931625,Bring the Flood's Debut FULL LENGTH ALBUM,Rock,Music,USD,2013-11-04,4500.0,2013-10-05 01:52:18,550.0,failed,14,US,550.00,550.00,4500.00,0,1
250597,344649785,New studio album from Darktown Jubilee!,Indie Rock,Music,GBP,2014-12-22,4000.0,2014-12-01 18:02:12,4081.0,successful,45,GB,6386.18,6340.21,6214.36,1,0
117841,159875043,Capture: Bible Edition,Games,Games,USD,2015-05-24,10000.0,2015-04-24 20:07:30,11.0,failed,2,US,11.00,11.00,10000.00,0,1
158905,1808267612,A Novel That Won't Quit Needs One More Boost,Fiction,Publishing,USD,2011-06-22,12000.0,2011-05-23 17:36:01,12700.0,successful,98,US,12700.00,12700.00,12000.00,1,1


Next, create a linear interpolation of `usd_pledged_real` as a function of the `backers`. Create a function `f` that generates a linear interpolation of `usd_pledged_real` as predicted by the amount of `backers`.

In [14]:
# Your code here:

f = interp1d(kick_sample['usd_pledged_real'],kick_sample['backers'])
f

Now create a new variable called `x_new`. This variable will contain all integers between the minimum number of backers in our sample and the maximum number of backers. The goal here is to take the dataset that contains few obeservations due to sampling and fill all observations with a value using the interpolation function. 

Hint: one option is the `np.arange` function.

In [17]:
# Your code here:
x_new = np.arange(kick_sample['backers'].min(), kick_sample['backers'].max()+1)
x_new


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98])

Plot function `f` for all values of `x_new`. Run the code below.

In [19]:
# Run this code:

%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(x_new, f(x_new))

##ValueError: A value in x_new is below the interpolation range.

ValueError: A value in x_new is below the interpolation range.

Next create a function that will generate a cubic interpolation function. Name the function `g`.

In [20]:
# Your code here:
g= interp1d(kick_sample['usd_pledged_real'], kick_sample['backers'],kind='cubic')
g


In [21]:
# Run this code:

plt.plot(x_new, g(x_new))

##ValueError: A value in x_new is below the interpolation ran

ValueError: A value in x_new is below the interpolation range.

# Bonus Challenge - The Binomial Distribution

The binomial distribution allows us to calculate the probability of k successes in n trials for a random variable with two possible outcomes (which we typically label success and failure).  

The probability of success is typically denoted by p and the probability of failure is denoted by 1-p.

The `scipy.stats` submodule contains a `binom` function for computing the probabilites of a random variable with the binomial distribution. You may read more about the binomial distribution [here](http://b.link/binomial55)

* In the cell below, compute the probability that a dice lands on 5 exactly 3 times in 8 tries.


In [ ]:
# Your code here:



* Do a simulation for the last event: do a function that simulate 8 tries and return a 1 if the result is 5 exactly 3 times and 0 if not. Now launch your simulation.

In [ ]:
# Your code here:


* Launch 10 simulations and represent the result in a bar plot. Now launch 1000 simulations and represent it. What do you see?

In [ ]:
# Your code here:
